<a href="https://colab.research.google.com/github/JunPark13/AIB_15_Section1_Project_-/blob/main/Part1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 확인하세요!

### 제출 전에 모든 코드가 순차적으로 잘 작동하는지 확인하세요. 
- **커널을 다시 시작**한 다음(메뉴바에서 Kernel$\rightarrow$Restart 선택) **모든 셀을 실행**해 보세요(메뉴바에서 Cell$\rightarrow$Run All 선택).

### 지정 셀의 지정 영역에서만 코드를 작성했는지 확인하세요.
- `YOUR CODE HERE`이 적혀 있는 셀에 대한 코드만 채점에 반영됩니다.
- `YOUR CODE HERE` ~ `YOUR CODE ENDS HERE` 사이에 본인의 코드를 작성하세요.
- 지정 셀 이외에 본인이 코드 실행 결과를 확인하기 위한 용도로 셀을 추가해도 됩니다.
    - **제출 시 지정 셀에서는 정답 이외에 결과 확인을 위한 코드 등은 제거하고 제출해 주세요!**
    - **기존 셀을 삭제하지 마세요! 채점에 불이익이 있을 수 있습니다.**

### 제출 방법
- 시간 내에 문제를 풀고, 본인의 github repo에 push 하세요.
    - **디렉토리 구조가 변하면 안 됩니다.** `codestates/ds-sc2xx`와 디렉토리 구조가 동일해야 합니다. 답안 파일의 이름을 변경하지 않았는지, 기존에 없던 폴더가 추가되지 않았는지 등을 확인하세요.
    - **코드스테이츠 가입 시 연동한 github 계정을 사용해야 합니다.**
    - **branch를 변경하여 작업하지 마세요.**
        - `master` branch에의 제출 사항에 대해서만 평가합니다.
        - 본인의 github repo `master` branch에 제출이 잘 되어 있는지 확인하세요.

---

# SC22x
 
### 코비 브라이언트의 슛 성공 여부 예측해보기

이번 스프린트 챌린지에서는 故 코비 브라이언트 선수의 NBA 필드 골 시도들이 담긴 데이터셋을 다룹니다 (1996년 11월 3일부터 2016년 4월 13일까지의 정규 시즌과 플레이오프 시즌 기록).

여러분은 각 슛의 성공 여부를 슈팅의 대한 정보, 게임에 대한 정보를 통해서 예측해야 합니다. 이것은 상당히 어려운 예측입니다. 해당 데이터셋은 [NBA API]((https://github.com/swar/nba_api) 파이썬 라이브러리를 통해 수집 됐습니다. 

In [ ]:
import sys
import warnings

!pip3 install xgboost

if "google.colab" in sys.modules:
    ## Colab에서 실행하는 경우 필요한 라이브러리를 미리 설치합니다.
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

warnings.filterwarnings("ignore")

In [91]:
# 데이터셋을 불러옵니다
import pandas as pd

url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/nba/nba_kobe_bryant.csv"
df = pd.read_csv(url)

# 데이터셋이 잘 불러졌는지 확인합니다
assert df.shape == (30697, 19)

In [92]:
df.shape

(30697, 19)

In [93]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season


In [94]:
# 살펴보기
df.isnull().sum()

game_id              0
game_event_id        0
player_name          0
period               0
minutes_remaining    0
seconds_remaining    0
action_type          0
shot_type            0
shot_zone_basic      0
shot_zone_area       0
shot_zone_range      0
shot_distance        0
loc_x                0
loc_y                0
shot_made_flag       0
game_date            0
htm                  0
vtm                  0
season_type          0
dtype: int64

# Part 1 - Data Preprocessing

## 1.1 분류를 위한 베이스라인 정확도를 계산하는 `calculate_baseline_score` 함수를 작성하세요.
- 예측해야 하는 target은 `shot_made_flag` 입니다. 
- 전체 데이터를 써서 baseline **accuracy score**를 계산하세요.
- 최빈 클래스를 사용하는 baseline을 가정하세요.

In [136]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

def calculate_baseline_score(df):
    accuracy_score = 0

    # YOUR CODE HERE
    target = 'shot_made_flag'
    features = df.drop(columns=[target])
    base = df[target].mode()[0]
    baseline = len(df[target]) * [base]
    cm = confusion_matrix(df[target], baseline, labels=[1, 0])
    accuracy_score = (cm[0, 0] + cm[1, 1]) / (cm[0, 0] + cm[0, 1] + cm[1, 0] + cm[1, 1])
    
    # YOUR CODE ENDS HERE

    return accuracy_score

In [137]:
baseline_accuracy_score = calculate_baseline_score(df)

In [138]:
print("baseline accuracy score:", baseline_accuracy_score)

baseline accuracy score: 0.5526924455158485


## 1.2 데이터셋을 아래의 기준을 사용해서 train/test 셋으로 나누는 `split_data` 함수를 작성하세요.

- `test set`: **2014-15** ~ **2015-16** 시즌
- NBA 시즌은 **10월**에 시작해서 **6월**에 종료합니다. 

In [98]:
def split_data(df):
    train = None
    test = None

    # YOUR CODE HERE
    train_set = df['game_date'] <= '2014-06-30'
    test_set = (df['game_date'] >= '2014-10-01') & (df['game_date'] <= '2016-06-30')
    train = df.loc[train_set]
    test = df.loc[test_set]

    # YOUR CODE ENDS HERE

    return train, test

In [99]:
train, test = split_data(df.copy())

In [100]:
train

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28866,21300366,34,Kobe Bryant,1,7,45,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,2,0,1,2013-12-17,MEM,LAL,Regular Season
28867,21300366,16,Kobe Bryant,1,10,9,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Right Side(R),8-16 ft.,11,106,50,1,2013-12-17,MEM,LAL,Regular Season
28868,21300366,5,Kobe Bryant,1,11,13,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,1,0,2013-12-17,MEM,LAL,Regular Season
28869,21300366,2,Kobe Bryant,1,11,41,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,22,-130,183,0,2013-12-17,MEM,LAL,Regular Season


## 1.3 새로운 feature를 만드는 `feature_engineering` 함수를 작성하세요.

- 새로운 이름을 갖는 최소 2개 이상의 새로운 feature를 만드세요. 아래의 리스트를 사용하셔도 되고 스스로의 아이디어로 만들어 주셔도 괜찮습니다.
    - **Homecourt Advantage**: 홈 팀(`htm`)이 Los Angeles Lakers (`LAL`)인지 여부
    - **Opponent**: LA Lakers를 상대하는 팀을 나타내는 feature
    - **Seconds remaining in the period**: 피리어드에 남은 총 시간 (분과 초를 합칩니다)
    - **Seconds remaining in the game**: 게임에 남은 총 시간 (피리어드와 초를 합칩니다). 농구에서 게임은 총 4개의 피리어드로 이루어져 있습니다 (각 12분)
    - **Made previous shot**: 코비 브라이언트의 예전 슛이 성공했는지 여부
- 학습에 사용하지 않는 특성은 drop하세요.
    

In [106]:
def feature_engineering(df):
    df_tmp = df.copy()

    # YOUR CODE HERE
    Homecourt_Advantage = []
    for i in df_tmp['htm'] :
        if i == 'LAL' :
            Homecourt_Advantage.append(1)
        else :
            Homecourt_Advantage.append(0)
    df_tmp['Homecourt Advantage'] = Homecourt_Advantage


    df_tmp['Change_m_to_s'] = df_tmp['minutes_remaining'] * 60
    df_tmp["Seconds remaining in the period"] = df_tmp['Change_m_to_s'] + df_tmp['seconds_remaining']

    df_tmp.drop(['game_event_id', 'player_name', 'htm', 'vtm', 'Change_m_to_s', 'seconds_remaining', 'minutes_remaining'], axis=1, inplace=True)


    # YOUR CODE ENDS HERE

    return df_tmp

In [107]:
train_processed = feature_engineering(train)
test_processed = feature_engineering(test)

In [108]:
train_processed.head()

,game_id,period,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,season_type,Homecourt Advantage,Seconds remaining in the period
0,29600027,1,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,Regular Season,1,42
1,29600031,2,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,Regular Season,0,608
2,29600044,2,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,Regular Season,0,517
3,29600044,2,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,Regular Season,0,394
4,29600044,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,Regular Season,0,327


In [109]:
train_processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28871 entries, 0 to 28870
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   game_id                          28871 non-null  int64 
 1   period                           28871 non-null  int64 
 2   action_type                      28871 non-null  object
 3   shot_type                        28871 non-null  object
 4   shot_zone_basic                  28871 non-null  object
 5   shot_zone_area                   28871 non-null  object
 6   shot_zone_range                  28871 non-null  object
 7   shot_distance                    28871 non-null  int64 
 8   loc_x                            28871 non-null  int64 
 9   loc_y                            28871 non-null  int64 
 10  shot_made_flag                   28871 non-null  int64 
 11  game_date                        28871 non-null  object
 12  season_type                     

In [110]:
test_processed.head()

,game_id,period,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,season_type,Homecourt Advantage,Seconds remaining in the period
28871,21400003,1,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,20,-54,201,0,2014-10-28,Regular Season,1,373
28872,21400003,1,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,-16,63,1,2014-10-28,Regular Season,1,285
28873,21400003,3,Turnaround Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),16-24 ft.,18,166,78,0,2014-10-28,Regular Season,1,352
28874,21400003,4,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,27,-138,241,0,2014-10-28,Regular Season,1,576
28875,21400003,4,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-179,88,0,2014-10-28,Regular Season,1,436


## 1.4 train / test 셋을 features(X)와 target(y)으로 각각 나누는 `divide_data` 함수를 작성하세요.
- `pd.DataFrame` 형태의 `X`와 `pd.Series` 형태의 `y`를 반환하세요.

In [121]:
def divide_data(df):
    X = None
    y = None

    # YOUR CODE HERE
    target = 'shot_made_flag'
    features = ['game_id', 'period', 'action_type', 'shot_type', 'shot_zone_basic','shot_zone_area', 
                'shot_zone_range', 'shot_distance', 'loc_x', 'loc_y', 'game_date', 'season_type', 
                'Homecourt Advantage', 'Seconds remaining in the period']
    X = df[features]
    y = df[target]
    
    # YOUR CODE ENDS HERE

    return X, y

In [122]:
X_train, y_train = divide_data(train_processed)
X_test, y_test = divide_data(test_processed)

In [123]:
X_train.shape, X_test.shape

((28871, 14), (1826, 14))

# Part 2 - Modeling & Evaluation

## 2.1 RandomForest 또는 XGBoost 모델 학습 파이프라인을 생성하고 학습하는 `fit_trees` 함수를 작성하세요. 
- Scikit-learn의 `pipeline`을 활용하여 범주형 feature의 encoder, imputer, 모델을 포함하는 하나의 학습 파이프라인을 구축하세요.
- 구축한 파이프라인에 대해 `RandomizedSearchCV`를 사용해 CV를 진행하세요.
    - 적어도 두 개 이상의 하이퍼파라미터에 대해 탐색을 진행하세요.
    - F1 score을 최대화하는 하이퍼파라미터 조합을 선택하세요.
- RandomForestClassifier, RandomizedSearchCV 등 랜덤성이 포함되는 방법론에 대해 모두 `random_state=42`로 하세요.
- cv는 5 이하, n_iter은 10회 이하로 하세요.

In [155]:
import sklearn
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder

### IMPORT ANY
# YOUR CODE HERE
from scipy.stats.distributions import uniform
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
# YOUR CODE ENDS HERE


def fit_trees(X_train, y_train):
    pipe = None    # pipe = make_pipeline(...)
    clf = None     # clf = RandomizedSearchCV(pipe, ...)

    # YOUR CODE HERE
    pipe = make_pipeline(
        OrdinalEncoder(),
        SimpleImputer(strategy="median"), 
        RandomForestClassifier(random_state=42))
    
    params = {
        "simpleimputer__strategy": ["median", "mean"],
        "randomforestclassifier__max_depth": [2, 4, 6],
        "randomforestclassifier__n_estimators": randint(50, 500),
        "randomforestclassifier__max_features": uniform(0, 1)}

    clf = RandomizedSearchCV(
        pipe,
        param_distributions=params,
        scoring="f1_score",
        n_iter=7,
        cv=3,
        verbose=3,
        random_state=42)
    
    # YOUR CODE ENDS HERE

    return clf

In [156]:
clf = fit_trees(X_train, y_train)

In [157]:
print("Best CV F1", clf.best_score_)
print("Best CV params", clf.best_params_)

AttributeError: ignored

## 2.2 학습된 모델로부터 feature importance를 계산하는 `get_feature_importances` 함수를 작성하세요. 
- feature_importances 변수에 feature_importances_ 값을 할당하면 됩니다.

In [ ]:
import numpy as np


def get_feature_importances(pipe, feature_names):
    feature_importances = np.zeros(len(feature_names))

    # YOUR CODE HERE
    
    
    # YOUR CODE ENDS HERE

    return sorted(
        list(zip(feature_names, feature_importances)), key=lambda x: x[1], reverse=True
    )

In [ ]:
feature_importances = get_feature_importances(
    clf.best_estimator_, list(X_train.columns)
)

In [ ]:
for name, importance in feature_importances:
    print(f"{name}: {importance:.4f}")

## 2.3 본인이 만든 모델의 Test 셋에서의 성능을 확인해 보세요.
- ### test_accuracy > 0.6, test_f1 > 0.5를 만족하면 기본 성능점수를 얻습니다.
- ###  (도전과제) test_accuracy > 0.68, test_f1 > 0.54을 만족하면 성능점수 만점을 얻습니다.
  - 도전과제까지 수행해야 스챌 3점을 얻습니다. 
  - 1.3으로 돌아가 다양한 특성 조합을 시도해 보세요. 
    - 그리고 2.2로 돌아와 피쳐 중요도를 계산해 보세요.
    - 어떤 피쳐의 중요도가 가장 높나요?
    - 중요도가 낮은 피쳐라도 있는 게 좋을까요?
  - 2.1로 돌아가 SearchCV에서 다양한 하이퍼파라미터 옵션 조합을 선택해 보세요.
    - 본인이 선택한 모델에서, 각 하이퍼파라미터가 어떤 의미를 가지나요?
    - SearchCV가 더 다양한 하이퍼파라미터 조합을 시도하도록 하기 위해 SearchCV의 어떤 옵션을 조정해야 할까요?

In [ ]:
def calculate_score(pipe, X_test, y_test):
    test_accuracy = pipe.score(X_test, y_test)
    test_f1 = f1_score(y_test, pipe.predict(X_test))
    return test_accuracy, test_f1

In [ ]:
test_accuracy, test_f1 = calculate_score(clf.best_estimator_, X_test, y_test)

In [ ]:
print("test accuracy: ", test_accuracy)
print("test F1:", test_f1)

In [ ]:
print("test accuracy > 0.68?: ", test_accuracy > 0.68)
print("test F1 > 0.54?:", test_f1 > 0.54)


---

# 수고하셨습니다!